In [36]:
%cd /content/drive/MyDrive/Project/Furniture_Image_Recognition

/content/drive/MyDrive/Project/Furniture_Image_Recognition


In [37]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [38]:
!pip install tensorflow pillow

In [39]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

In [40]:
IMG_SIZE = 150
BATCH_SIZE = 32
EPOCHS = 20


In [41]:
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [42]:
train_generator = datagen.flow_from_directory(
    'furniture-dataset/classes',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

Found 434 images belonging to 7 classes.


In [43]:
validation_generator = datagen.flow_from_directory(
    'furniture-dataset/classes',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

Found 107 images belonging to 7 classes.


In [44]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(len(train_generator.class_indices), activation='softmax')
])

In [45]:
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [46]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE,
    epochs=EPOCHS
)

Epoch 1/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 52s 3s/step - accuracy: 0.1531 - loss: 2.5492 - val_accuracy: 0.1979 - val_loss: 1.7909
Epoch 2/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 6s 251ms/step - accuracy: 0.2812 - loss: 1.7796 - val_accuracy: 0.3636 - val_loss: 1.7124
Epoch 3/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 73s 3s/step - accuracy: 0.2489 - loss: 1.7643 - val_accuracy: 0.4062 - val_loss: 1.4891
Epoch 4/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.3750 - loss: 1.5423 - val_accuracy: 0.2727 - val_loss: 1.7473
Epoch 5/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 43s 3s/step - accuracy: 0.3970 - loss: 1.4666 - val_accuracy: 0.4479 - val_loss: 1.4016
Epoch 6/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - accuracy: 0.4444 - loss: 1.4417 - val_accuracy: 0.1818 - val_loss: 2.2822
Epoch 7/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 47s 3s/step - accuracy: 0.4201 - loss: 1.5175 - val_accuracy: 0.5000 - val_loss: 1.3290
Epoch 8/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - accuracy: 0.5000 - loss: 1.3874 - val_accuracy: 0.5455 - val_

In [47]:
model.save('furniture_recognition_model.h5')

In [48]:
import json
class_labels = list(train_generator.class_indices.keys())
with open('class_labels.json', 'w') as f:
    json.dump(class_labels, f)